In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split

2024-03-03 20:11:57.650134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 20:11:58.258279: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-03 20:11:58.258351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-03 20:11:58.258356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import relevant datasets
X_train = pd.read_csv("finalXtrain.csv")
y_train = pd.read_csv("finalYtrain.csv")
X_test = pd.read_csv("finalXtest.csv")
y_test = pd.read_csv("finalYtest.csv")
X_val = pd.read_csv("finalXval.csv")
y_val = pd.read_csv("finalYval.csv")

In [4]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(221, 72)
(221, 3)
(47, 72)
(47, 3)
(48, 72)
(48, 3)


In [5]:
from tensorflow.keras.models import load_model
pre_trained_model = load_model('bestmodeloct9/alaarg_oct9.h5')

2024-03-03 20:11:58.732172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 20:11:58.732494: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 20:11:58.738775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 20:11:58.738983: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-03 20:11:58.739147: I tensorflow/compiler/xla/stream_executo

In [6]:
print(pre_trained_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 15)            60        
                                                                 
 batch_normalization (BatchN  (None, 24, 15)           60        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 15)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 15)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 30)             2280  

In [8]:
for layer in pre_trained_model.layers[:-1]:
    print(layer)
    layer.trainable = False

In [9]:
x = pre_trained_model.layers[-2].output 

In [10]:
from tensorflow.keras.models import Model

# new_output = Dense(3, activation='softmax', name="newout")(x)
# x = Dropout(0.5,seed=seed_value)(x)
# x = Dense(20,activation ='sigmoid')(x)
# x = Dense(10,activation ='sigmoid')(x)
# x = Dense(5,activation ='tanh')(x)
outputs = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=pre_trained_model.input, outputs=outputs)

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 15)            60        
                                                                 
 batch_normalization (BatchN  (None, 24, 15)           60        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 15)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 15)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 30)             2280  

In [13]:
#earlystopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=100)
checkpoint_path = 'alaargly_transferfinallayer.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

In [14]:

history = model.fit(X_train, y_train,
              batch_size=5,
              verbose=1,
              epochs=400,
              validation_data = (X_val,y_val), 
              callbacks = [es,model_checkpoint]
                    ,class_weight = {0:2.4,1:2.23,2:7.37}
                   )

Epoch 1/400


2024-03-03 20:12:01.481818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2024-03-03 20:12:01.657806: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 1/45 [..............................] - ETA: 1:06 - loss: 4.0202 - accuracy: 0.0000e+00

2024-03-03 20:12:01.694995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-03 20:12:01.696991: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f5b8ea10eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-03 20:12:01.697002: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-03 20:12:01.697006: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-03 20:12:01.700971: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-03 20:12:01.760027: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
20

25/45 [===============>..............] - ETA: 0s - loss: 4.5914 - accuracy: 0.1520      
Epoch 1: val_accuracy improved from -inf to 0.25000, saving model to alaargly_transferfinallayer.h5
45/45 [==============================] - 2s 7ms/step - loss: 4.8133 - accuracy: 0.1629 - val_loss: 1.5957 - val_accuracy: 0.2500
Epoch 2/400
29/45 [==================>...........] - ETA: 0s - loss: 4.1255 - accuracy: 0.1862
Epoch 2: val_accuracy improved from 0.25000 to 0.29167, saving model to alaargly_transferfinallayer.h5
45/45 [==============================] - 0s 3ms/step - loss: 4.2400 - accuracy: 0.1900 - val_loss: 1.4218 - val_accuracy: 0.2917
Epoch 3/400
32/45 [====================>.........] - ETA: 0s - loss: 3.5212 - accuracy: 0.1937
Epoch 3: val_accuracy did not improve from 0.29167
45/45 [==============================] - 0s 3ms/step - loss: 3.8552 - accuracy: 0.1855 - val_loss: 1.3061 - val_accuracy: 0.2500
Epoch 4/400
32/45 [====================>.........] - ETA: 0s - loss: 3.7186 - ac

Epoch 30/400
31/45 [===================>..........] - ETA: 0s - loss: 2.1513 - accuracy: 0.7677
Epoch 30: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.1340 - accuracy: 0.7557 - val_loss: 1.0268 - val_accuracy: 0.5208
Epoch 31/400
31/45 [===================>..........] - ETA: 0s - loss: 2.3416 - accuracy: 0.7097
Epoch 31: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.2647 - accuracy: 0.7240 - val_loss: 1.0225 - val_accuracy: 0.5417
Epoch 32/400
28/45 [=================>............] - ETA: 0s - loss: 2.6185 - accuracy: 0.7000
Epoch 32: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.4653 - accuracy: 0.7014 - val_loss: 1.0249 - val_accuracy: 0.5417
Epoch 33/400
33/45 [=====================>........] - ETA: 0s - loss: 2.2807 - accuracy: 0.6909
Epoch 33: val_accuracy did not improve from 0.64583
45/45 [==============

Epoch 60/400
31/45 [===================>..........] - ETA: 0s - loss: 2.3054 - accuracy: 0.6581
Epoch 60: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.3751 - accuracy: 0.6606 - val_loss: 1.0502 - val_accuracy: 0.5208
Epoch 61/400
31/45 [===================>..........] - ETA: 0s - loss: 2.2541 - accuracy: 0.7548
Epoch 61: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.1971 - accuracy: 0.7421 - val_loss: 1.0539 - val_accuracy: 0.5208
Epoch 62/400
31/45 [===================>..........] - ETA: 0s - loss: 2.1432 - accuracy: 0.6710
Epoch 62: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.4608 - accuracy: 0.6516 - val_loss: 1.0460 - val_accuracy: 0.5208
Epoch 63/400
31/45 [===================>..........] - ETA: 0s - loss: 2.2221 - accuracy: 0.6839
Epoch 63: val_accuracy did not improve from 0.64583
45/45 [==============

Epoch 90/400
31/45 [===================>..........] - ETA: 0s - loss: 2.3776 - accuracy: 0.6645
Epoch 90: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.2982 - accuracy: 0.6833 - val_loss: 1.0737 - val_accuracy: 0.5208
Epoch 91/400
31/45 [===================>..........] - ETA: 0s - loss: 1.9388 - accuracy: 0.7355
Epoch 91: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.2685 - accuracy: 0.7195 - val_loss: 1.0708 - val_accuracy: 0.5208
Epoch 92/400
31/45 [===================>..........] - ETA: 0s - loss: 2.2490 - accuracy: 0.7161
Epoch 92: val_accuracy did not improve from 0.64583
45/45 [==============================] - 0s 3ms/step - loss: 2.2544 - accuracy: 0.7149 - val_loss: 1.0771 - val_accuracy: 0.5208
Epoch 93/400
32/45 [====================>.........] - ETA: 0s - loss: 2.2906 - accuracy: 0.7250
Epoch 93: val_accuracy did not improve from 0.64583
45/45 [==============

In [15]:
model1 = tf.keras.models.load_model('alaargly_transferfinallayer.h5')
model1.evaluate(X_test,y_test)

2/2 [==============================] - 0s 8ms/step - loss: 1.0314 - accuracy: 0.7021


[1.0314030647277832, 0.7021276354789734]

In [16]:
y=model1.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [17]:
#class-wise accuracy
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.to_numpy().argmax(axis=1), y.argmax(axis=1))
matrix.diagonal()/matrix.sum(axis=1)

array([0.9       , 0.71428571, 0.        ])

In [19]:
y_test[:2]

,L_Alanine,L_Arginine,L_Lysine
0,0,1,0
1,0,1,0
